In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import random

scaler = MinMaxScaler()

In [ ]:
### 데이터 불러오기
FREDMD = pd.read_excel("FREDMDdata.xlsx")

In [ ]:
### 정규화 함수

def normalize(df):
    return pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

def denormalize(val, minval, maxval):
    return val * (maxval - minval) + minval

In [ ]:
### 데이터 정규화

FREDMX_norm = normalize(FREDMD)

In [ ]:
y = FREDMX_norm['INDPRO']
X = FREDMX_norm.drop(columns=['INDPRO'])

In [8]:
n_features = len(X.T)
timestep = 4
horizon = 1

In [ ]:
### 3D-Array 생성

def create_dataset(x, y, timestep, horizon):
    x = np.asarray(x)
    y = np.asarray(y)

    n_samples = x.shape[0] - timestep - horizon + 1
    X = np.empty((n_samples, timestep, x.shape[1]))
    Y = y[timestep + horizon - 1:]

    for i in range(n_samples):
        X[i] = x[i:(i + timestep), :]

    return X, Y

def create_X_out(x, timestep):
    x = np.asarray(x)
    X_last = x[-timestep:] 
    X_last = X_last.reshape((1, timestep, x.shape[1]))
    return X_last

In [10]:
ds = create_dataset(X, y, timestep, horizon)

In [11]:
X_train, y_train = create_dataset(X, y, timestep=4, horizon=1)

In [12]:
X_out_arr = create_X_out(X, timestep)

In [ ]:
### Random seed 설정

random.seed(21)
np.random.seed(21)
tf.random.set_seed(21)

In [ ]:
### 단일층 LSTM

model = Sequential()
model.add(LSTM(units=32, input_shape=(timestep, n_features), stateful=False))
model.add(Dense(units=1))

In [15]:
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                19712     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 19,745
Trainable params: 19,745
Non-trainable params: 0
_________________________________________________________________


In [ ]:
### 모델 학습

model.fit(X_train, y_train, epochs=100, batch_size=50,shuffle=False,verbose=2)

Epoch 1/100
10/10 - 4s - loss: 0.1141 - 4s/epoch - 371ms/step
Epoch 2/100
10/10 - 0s - loss: 0.0217 - 35ms/epoch - 4ms/step
Epoch 3/100
10/10 - 0s - loss: 0.0171 - 33ms/epoch - 3ms/step
Epoch 4/100
10/10 - 0s - loss: 0.0124 - 33ms/epoch - 3ms/step
Epoch 5/100
10/10 - 0s - loss: 0.0122 - 35ms/epoch - 4ms/step
Epoch 6/100
10/10 - 0s - loss: 0.0111 - 36ms/epoch - 4ms/step
Epoch 7/100
10/10 - 0s - loss: 0.0111 - 37ms/epoch - 4ms/step
Epoch 8/100
10/10 - 0s - loss: 0.0108 - 43ms/epoch - 4ms/step
Epoch 9/100
10/10 - 0s - loss: 0.0106 - 37ms/epoch - 4ms/step
Epoch 10/100
10/10 - 0s - loss: 0.0105 - 33ms/epoch - 3ms/step
Epoch 11/100
10/10 - 0s - loss: 0.0104 - 35ms/epoch - 4ms/step
Epoch 12/100
10/10 - 0s - loss: 0.0103 - 36ms/epoch - 4ms/step
Epoch 13/100
10/10 - 0s - loss: 0.0102 - 35ms/epoch - 4ms/step
Epoch 14/100
10/10 - 0s - loss: 0.0102 - 34ms/epoch - 3ms/step
Epoch 15/100
10/10 - 0s - loss: 0.0101 - 34ms/epoch - 3ms/step
Epoch 16/100
10/10 - 0s - loss: 0.0100 - 33ms/epoch - 3ms/step
E

In [ ]:
### 예측

h_step_ahead_pred = model.predict(X_out_arr)

1/1 [==============================] - 1s 729ms/step


In [18]:
x_min = FREDMD.apply(min)
x_max = FREDMD.apply(max)

y_min = x_min['INDPRO']
y_max = x_max['INDPRO']

In [ ]:
### Denormalize하여 예측값 생성

y_pred = denormalize(h_step_ahead_pred, y_min, y_max)

In [20]:
y_pred

array([[-0.00100458]], dtype=float32)

In [ ]:
### 2-layer LSTM with dropout (rate = 0.3)

model2 = Sequential()
model2.add(LSTM(units=32, input_shape=(timestep, n_features),
                 return_sequences=True, stateful=False))
model2.add(Dropout(rate=0.3))
model2.add(LSTM(units=16))
model2.add(Dropout(rate=0.3))
model2.add(Dense(units=1))

In [22]:
model2.compile(loss='mse', optimizer='adam')
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 4, 32)             19712     
                                                                 
 dropout (Dropout)           (None, 4, 32)             0         
                                                                 
 lstm_2 (LSTM)               (None, 16)                3136      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 22,865
Trainable params: 22,865
Non-trainable params: 0
_________________________________________________________________


In [ ]:
### 모델 학습

model2.fit(X_train, y_train, epochs=100, batch_size=50, shuffle=False, verbose=2)

Epoch 1/100
10/10 - 6s - loss: 0.1428 - 6s/epoch - 628ms/step
Epoch 2/100
10/10 - 0s - loss: 0.0498 - 55ms/epoch - 6ms/step
Epoch 3/100
10/10 - 0s - loss: 0.0385 - 54ms/epoch - 5ms/step
Epoch 4/100
10/10 - 0s - loss: 0.0294 - 57ms/epoch - 6ms/step
Epoch 5/100
10/10 - 0s - loss: 0.0278 - 58ms/epoch - 6ms/step
Epoch 6/100
10/10 - 0s - loss: 0.0254 - 60ms/epoch - 6ms/step
Epoch 7/100
10/10 - 0s - loss: 0.0261 - 63ms/epoch - 6ms/step
Epoch 8/100
10/10 - 0s - loss: 0.0273 - 56ms/epoch - 6ms/step
Epoch 9/100
10/10 - 0s - loss: 0.0288 - 55ms/epoch - 6ms/step
Epoch 10/100
10/10 - 0s - loss: 0.0253 - 54ms/epoch - 5ms/step
Epoch 11/100
10/10 - 0s - loss: 0.0247 - 54ms/epoch - 5ms/step
Epoch 12/100
10/10 - 0s - loss: 0.0232 - 54ms/epoch - 5ms/step
Epoch 13/100
10/10 - 0s - loss: 0.0208 - 57ms/epoch - 6ms/step
Epoch 14/100
10/10 - 0s - loss: 0.0229 - 61ms/epoch - 6ms/step
Epoch 15/100
10/10 - 0s - loss: 0.0233 - 65ms/epoch - 7ms/step
Epoch 16/100
10/10 - 0s - loss: 0.0220 - 65ms/epoch - 7ms/step
E

In [ ]:
### 예측

h_step_ahead_pred2 = model2.predict(X_out_arr)

1/1 [==============================] - 1s 1s/step


In [ ]:
### Denormalize하여 예측값 생성

y_pred2 = denormalize(h_step_ahead_pred2, y_min, y_max)

In [26]:
h_step_ahead_pred2

array([[0.53835905]], dtype=float32)